In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint
from scipy.signal import decimate

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)
layers = []

In [ ]:
prms = gp.NuzzleParams()
prms.dilate_multiplier_max=0.15
prms.dilate_cap_style = 2
prms.dilate_join_style = 2
prms.erode_cap_style = 2
prms.erode_join_style =2

In [ ]:
min_rad = 1.5
iter_max = 50

In [ ]:
all_circles = []
circle_to_fill = drawbox


In [ ]:
n_fails = 0
n_iters = 0
while (n_fails < 10) and (n_iters < iter_max):
    try:
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        rads = np.linspace(fc_rad*0.5, fc_rad*0.07, 15)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        if circles.type == 'MultiPolygon':
            circles =gp.nuzzle_em(circles, prms, n_iters=20)
            circle_to_fill.filled = True
            circles = circles.intersection(circle_to_fill)
            all_circles += [c.buffer(-0.) for c in circles]
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
MultiPolygon(all_circles).boundary

In [ ]:
bottom_level_circles = [c for c in all_circles if not getattr(c, 'filled', False)]

In [ ]:
blcb = MultiPolygon(bottom_level_circles).buffer(-0.5).boundary

In [ ]:
fill_circles = []
for c in all_circles:
    if not getattr(c, 'filled', False):
        
        rad = gp.get_rad(c)
        if rad > 0.5:
            _c = c.buffer(-0.5)
            spacing = np.random.uniform(-0.5, -0.51)
            erodes = np.arange(spacing, -rad+0.5, spacing)
    #         erodes *= (np.random.randn(*erodes.shape) * 0.01 + 1)
            fill_circles += [_c.buffer(b) for b in erodes]

fills = MultiPolygon(fill_circles).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('1mm')



sk.stroke(1)
sk.geometry(blcb)
sk.stroke(2)
sk.geometry(fills)

for tol in [0.1]:
    sk.vpype(f'linesimplify -t {tol}mm linemerge -t {tol}mm')

sk.vpype('linesort')

sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/258_any_shape_none_touch.svg')

# try 3

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)
layers = []

In [ ]:
prms = gp.NuzzleParams()
prms.dilate_multiplier_max=0.15
prms.dilate_cap_style = 2
prms.dilate_join_style = 2
prms.erode_cap_style = 2
prms.erode_join_style =2

In [ ]:
min_rad = 1.5
iter_max = 10

In [ ]:
all_circles = []
circle_to_fill = drawbox


In [ ]:
n_fails = 0
n_iters = 0
while (n_fails < 10) and (n_iters < iter_max):
    try:
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        rads = np.linspace(fc_rad*0.5, fc_rad*0.07, 15)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        if circles.type == 'MultiPolygon':
            circles =gp.nuzzle_em(circles, prms, n_iters=20)
            circle_to_fill.filled = True
            circles = circles.intersection(circle_to_fill)
            all_circles += [c.buffer(-0.) for c in circles]
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
MultiPolygon(all_circles).boundary

In [ ]:
bottom_level_circles = [c for c in all_circles if not getattr(c, 'filled', False)]

In [ ]:
blcb = MultiPolygon(bottom_level_circles).buffer(-0.5).boundary

In [ ]:
fill_circles = []
for c in all_circles:
    if not getattr(c, 'filled', False):
        
        rad = gp.get_rad(c)
        if rad > 0.5:
            _c = c.buffer(-0.5)
            spacing = np.random.uniform(-0.5, -0.8)
            erodes = np.arange(spacing, -rad+0.5, spacing)
    #         erodes *= (np.random.randn(*erodes.shape) * 0.01 + 1)
            fill_circles += [_c.buffer(b) for b in erodes]

fills = MultiPolygon(fill_circles).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('1mm')



sk.stroke(1)
sk.geometry(blcb)
sk.stroke(2)
sk.geometry(fills)

for tol in [0.1]:
    sk.vpype(f'linesimplify -t {tol}mm linemerge -t {tol}mm')

sk.vpype('linesort')

sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/259_any_shape_none_touch_flaws.svg')

# try 4

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)
layers = []

In [ ]:
prms = gp.NuzzleParams()
prms.dilate_multiplier_max=0.15
prms.dilate_cap_style = 2
prms.dilate_join_style = 2
prms.erode_cap_style = 2
prms.erode_join_style =2

In [ ]:
min_rad = 1.5
iter_max = 1

In [ ]:
all_circles = []
circle_to_fill = drawbox


In [ ]:
n_fails = 0
n_iters = 0
while (n_fails < 10) and (n_iters < iter_max):
    try:
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        rads = np.linspace(fc_rad*0.5, fc_rad*0.3, 15)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        if circles.type == 'MultiPolygon':
            circles =gp.nuzzle_em(circles, prms, n_iters=20)
            circle_to_fill.filled = True
            circles = circles.intersection(circle_to_fill)
            all_circles += [c.buffer(-0.) for c in circles]
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
MultiPolygon(all_circles).boundary

In [ ]:
bottom_level_circles = [c for c in all_circles if not getattr(c, 'filled', False)]

In [ ]:
blcb = MultiPolygon(bottom_level_circles).buffer(-0.5).boundary

In [ ]:
fill_circles = []
for c in all_circles:
    if not getattr(c, 'filled', False):
        
        rad = gp.get_rad(c)
        if rad > 0.5:
            _c = c.buffer(-0.5)
            spacing = np.random.uniform(-0.5, -1)
            erodes = np.arange(spacing, -rad+0.5, spacing)
    #         erodes *= (np.random.randn(*erodes.shape) * 0.01 + 1)
            fill_circles += [_c.buffer(b) for b in erodes]

fills = MultiPolygon(fill_circles).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('1mm')



sk.stroke(1)
sk.geometry(blcb)
sk.stroke(2)
sk.geometry(fills)

for tol in [0.2, 0.4, 0.8, 1.]:
    sk.vpype(f'linesimplify -t {tol}mm linemerge -t {tol}mm')

sk.vpype('linesort')

sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/260_any_shape_none_touch_flaws.svg')